In [3]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import csv
import re
import concurrent.futures

**Reading in the Data**

In [249]:
# part 1
r = requests.get("http://www.ufcstats.com/statistics/events/completed?page=all")
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_ = 'b-statistics__section' )
pages = s.find_all('a', class_ = 'b-link b-link_style_black')
hrefs_start = [a['href'] for a in pages]
hrefs_start = hrefs_start[0:550]

stats_url = []
fighter1=[]
fighter2=[]
weight = []
method =[]
rounds =[]
times = []
dates = []
locations = []
event = []

def get_event_details(url):
        time.sleep(1)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')
        rows = rows[1:]
        for row in rows:
            cols = row.find_all('td')
            data = [col.get_text(strip=True) for col in cols][6:]
            weight.append(data[0])
            method.append(data[1])
            rounds.append(data[2])
            times.append(data[3])
            fighters = row.find_all('a', class_ ="b-link b-link_style_black")
            names = [a.get_text(strip = True) for a in fighters]
            fighter1.append(names[0])
            fighter2.append(names[1])
            stat_link = row.get('data-link')
            
            if stat_link != '' or stat_link is not None:
                stats_url.append(row.get('data-link'))
            else: stats_url.append("No Link")

            location = soup.find_all('li', class_ = 'b-list__box-list-item')
            locations.append(location[1].get_text(strip=True)[9:])
            date = soup.find('li', class_ = 'b-list__box-list-item')
            dates.append(date.get_text(strip=True)[5:])
            event.append(soup.find('h2').get_text(strip=True))


with concurrent.futures.ThreadPoolExecutor() as executor:
     executor.map(get_event_details, hrefs_start)

In [250]:
# part 2
r = requests.get("http://www.ufcstats.com/statistics/events/completed?page=all")
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_ = 'b-statistics__section' )
pages = s.find_all('a', class_ = 'b-link b-link_style_black')
hrefs_start = [a['href'] for a in pages]
hrefs_start = hrefs_start[550:]

stats_url1 = []
fighter11 =[]
fighter21 =[]
weight1 = []
method1 =[]
rounds1 =[]
times1 = []
dates1 = []
locations1 = []
event1=[]

def get_event_details(url):
        time.sleep(1)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')
        rows = rows[1:]
        for row in rows:
            cols = row.find_all('td')
            data = [col.get_text(strip=True) for col in cols][6:]
            weight1.append(data[0])
            method1.append(data[1])
            rounds1.append(data[2])
            times1.append(data[3])
            fighters = row.find_all('a', class_ ="b-link b-link_style_black")
            names = [a.get_text(strip = True) for a in fighters]
            fighter11.append(names[0])
            fighter21.append(names[1])
            stat_link = row.get('data-link')
            
            if stat_link != '' or stat_link is not None:
                stats_url1.append(row.get('data-link'))
            else: stats_url1.append("No Link")

            location = soup.find_all('li', class_ = 'b-list__box-list-item')
            locations1.append(location[1].get_text(strip=True)[9:])
            date = soup.find('li', class_ = 'b-list__box-list-item')
            dates1.append(date.get_text(strip=True)[5:])
            event1.append(soup.find('h2').get_text(strip=True))


with concurrent.futures.ThreadPoolExecutor() as executor:
     executor.map(get_event_details, hrefs_start)

In [1]:
print(len(fighter1))
print(len(stats_url))
print(len(event))

NameError: name 'fighter1' is not defined

In [252]:
df_1 = pd.DataFrame({
    "event": event,
    'fighter1': fighter1,
    'fighter2': fighter2,
    'weight': weight,
    'rounds': rounds,
    'times': times,
    'method': method,
    'locations': locations,
    'dates': dates,
    'stats_url': stats_url
})

In [253]:
df_2 = pd.DataFrame({
    "event": event1,
    'fighter1': fighter11,
    'fighter2': fighter21,
    'weight': weight1,
    'rounds': rounds1,
    'times': times1,
    'method': method1,
    'locations': locations1,
    'dates': dates1,
    'stats_url': stats_url1
})

In [254]:
df_1.head

<bound method NDFrame.head of                                           event           fighter1  \
0     UFC Fight Night: Machado Garry vs. Prates  Ian Machado Garry   
1     UFC Fight Night: Machado Garry vs. Prates     Zhang Mingyang   
2     UFC Fight Night: Machado Garry vs. Prates        David Onama   
3     UFC Fight Night: Machado Garry vs. Prates     Abus Magomedov   
4     UFC Fight Night: Machado Garry vs. Prates        Randy Brown   
...                                         ...                ...   
6119                    UFC 134: Silva vs Okami       Paulo Thiago   
6120                    UFC 134: Silva vs Okami   Raphael Assuncao   
6121                    UFC 134: Silva vs Okami        Erick Silva   
6122                    UFC 134: Silva vs Okami     Iuri Alcantara   
6123                    UFC 134: Silva vs Okami       Yves Jabouin   

            fighter2             weight rounds times        method  \
0      Carlos Prates       Welterweight      5  5:00       

In [256]:
df_1.to_csv('UFC_events_1.csv')
df_2.to_csv('UFC_events_2.csv')

In [257]:
final_df = pd.concat([df_1, df_2], ignore_index=True)

In [258]:
final_df.shape

(7613, 10)

In [ ]:
final_df.to_csv('Final_UFC_Events.csv')
#filter out methods that are odd (DQ's, NC's, basically anything other than a decisive victory for one competitor)


In [348]:
# Getting more in-depth stats - part 1
fight_details_links = final_df['stats_url']
# fight_details_links = fight_details_links[0:2000]

f1 = []
f2 = []
f1_kd = []
f1_sigstr = []
f1_sigstr_pct = []
f1_totstr = []
f1_td = []
f1_td_pct = []
f1_subatt = []
f1_rev = []
f1_ctrl = []
row2 = []
f2_kd = []
f2_sigstr = []
f2_sigstr_pct = []
f2_totstr = []
f2_td = []
f2_td_pct = []
f2_subatt = []
f2_rev = []
f2_ctrl = []
event = []
stats_url = []


connect_timeout = 6
read_timeout = 60
count = 0


for url in fight_details_links:
    time.sleep(3)
    r = requests.get(url, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(r.content, 'html.parser')
    fight_details = soup.find_all('section', class_ = 'b-fight-details__section js-fight-section')
    fight_details = fight_details[1]
    tds = fight_details.find_all('td')
    row1 = []
    event.append(soup.find('h2').get_text(strip=True))
    for td in tds:
        p_tags = td.find_all('p')
        row1.append(p_tags[0].get_text(strip=True))
    f1.append(row1[0])
    f1_kd.append(row1[1])
    f1_sigstr.append(row1[2])
    f1_sigstr_pct.append(row1[3])
    f1_totstr.append(row1[4])
    f1_td.append(row1[5])
    f1_td_pct.append(row1[6])
    f1_subatt.append(row1[7])
    f1_rev.append(row1[8])
    f1_ctrl.append(row1[9])
    row2 = []
    for td in tds:
        p_tags = td.find_all('p')
        row2.append(p_tags[1].get_text(strip=True))
    f2.append(row2[0])
    f2_kd.append(row2[1])
    f2_sigstr.append(row2[2])
    f2_sigstr_pct.append(row2[3])
    f2_totstr.append(row2[4])
    f2_td.append(row2[5])
    f2_td_pct.append(row2[6])
    f2_subatt.append(row2[7])
    f2_rev.append(row2[8])
    f2_ctrl.append(row2[9])
    stats_url.append(url)

    count += 1
    print(f"Succcess count: {count}")

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     executor.map(get_fight_stats, fight_details_links)

Succcess count: 1
Succcess count: 2
Succcess count: 3
Succcess count: 4
Succcess count: 5
Succcess count: 6
Succcess count: 7
Succcess count: 8
Succcess count: 9
Succcess count: 10
Succcess count: 11
Succcess count: 12
Succcess count: 13
Succcess count: 14
Succcess count: 15
Succcess count: 16
Succcess count: 17
Succcess count: 18
Succcess count: 19
Succcess count: 20
Succcess count: 21
Succcess count: 22
Succcess count: 23
Succcess count: 24
Succcess count: 25
Succcess count: 26
Succcess count: 27
Succcess count: 28
Succcess count: 29
Succcess count: 30
Succcess count: 31
Succcess count: 32
Succcess count: 33
Succcess count: 34
Succcess count: 35
Succcess count: 36
Succcess count: 37
Succcess count: 38
Succcess count: 39
Succcess count: 40
Succcess count: 41
Succcess count: 42
Succcess count: 43
Succcess count: 44
Succcess count: 45
Succcess count: 46
Succcess count: 47
Succcess count: 48
Succcess count: 49
Succcess count: 50
Succcess count: 51
Succcess count: 52
Succcess count: 53
Su

IndexError: list index out of range

In [345]:
print(fight_details_links[228])

http://www.ufcstats.com/fight-details/eba46dbf9cdd83c0


In [351]:
print(len(f1))
print(len(f2_sigstr))
print(len(event))
print(len(f1_rev))
print(len(stats_url))
print(len(stats_url[:-1]))
#stats_url = stats_url[:-1]

3152
3152
3152
3152
3153
3152


In [ ]:
# Getting more in-depth stats - part 2
fight_details_links = final_df['stats_url']
fight_details_links = fight_details_links[len(stats_url):]


connect_timeout = 6
read_timeout = 60
count = 0


for url in fight_details_links:
    time.sleep(3)
    r = requests.get(url, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(r.content, 'html.parser')
    fight_details = soup.find_all('section', class_ = 'b-fight-details__section js-fight-section')
    fight_details = fight_details[1]
    tds = fight_details.find_all('td')
    row1 = []
    event.append(soup.find('h2').get_text(strip=True))
    for td in tds:
        p_tags = td.find_all('p')
        row1.append(p_tags[0].get_text(strip=True))
    f1.append(row1[0])
    f1_kd.append(row1[1])
    f1_sigstr.append(row1[2])
    f1_sigstr_pct.append(row1[3])
    f1_totstr.append(row1[4])
    f1_td.append(row1[5])
    f1_td_pct.append(row1[6])
    f1_subatt.append(row1[7])
    f1_rev.append(row1[8])
    f1_ctrl.append(row1[9])
    row2 = []
    for td in tds:
        p_tags = td.find_all('p')
        row2.append(p_tags[1].get_text(strip=True))
    f2.append(row2[0])
    f2_kd.append(row2[1])
    f2_sigstr.append(row2[2])
    f2_sigstr_pct.append(row2[3])
    f2_totstr.append(row2[4])
    f2_td.append(row2[5])
    f2_td_pct.append(row2[6])
    f2_subatt.append(row2[7])
    f2_rev.append(row2[8])
    f2_ctrl.append(row2[9])
    stats_url.append(url)

    count += 1
    print(f"Succcess count: {count}")

In [367]:
stats_df1 = pd.DataFrame({
'fighter1' : f1,
'fighter2': f2,
'f1_kd' : f1_kd,
'f1_sigstr' : f1_sigstr,
'f1_sigstr_pct' : f1_sigstr_pct,
'f1_totstr' : f1_totstr,
'f1_td' : f1_td,
'f1_td_pct' : f1_td_pct,
'f1_subatt' : f1_subatt,
'f1_rev' : f1_rev,
'f1_ctrl' : f1_ctrl,
'f2_kd' : f2_kd,
'f2_sigstr' : f2_sigstr,
'f2_sigstr_pct' : f2_sigstr_pct,
'f2_totstr' : f2_totstr,
'f2_td' : f2_td,
'f2_td_pct' : f2_td_pct,
'f2_subatt' : f2_subatt,
'f2_rev' : f2_rev,
'f2_ctrl' : f2_ctrl,
'event' : event,
'stats_url' : stats_url
})

stats_df1.to_csv("Complete Stats.csv")

In [364]:
print(len(f1))

7433


In [283]:
final_df.shape

(7613, 10)

In [371]:
print(stats_df1['stats_url'])

0       http://www.ufcstats.com/fight-details/633f4b5e...
1       http://www.ufcstats.com/fight-details/a526a72a...
2       http://www.ufcstats.com/fight-details/4072abf0...
3       http://www.ufcstats.com/fight-details/9bbd4577...
4       http://www.ufcstats.com/fight-details/fbf83609...
                              ...                        
7428    http://www.ufcstats.com/fight-details/17526cb9...
7429    http://www.ufcstats.com/fight-details/1e14e79a...
7430    http://www.ufcstats.com/fight-details/74dcccd3...
7431    http://www.ufcstats.com/fight-details/8d833540...
7432    http://www.ufcstats.com/fight-details/21aa163b...
Name: stats_url, Length: 7433, dtype: object


In [395]:
final_df['method'].unique()

array(['U-DEC', 'KO/TKOElbows', 'KO/TKOPunch', 'KO/TKOPunches', 'S-DEC',
       'SUBRear Naked Choke', 'KO/TKOFlying Knee', 'KO/TKOKnee',
       'SUBTriangle Choke', 'KO/TKOKick', 'SUBArmbar', 'SUBArm Triangle',
       'KO/TKO', 'SUBKimura', 'SUBGuillotine Choke', "SUBD'Arce Choke",
       'KO/TKOSpinning Back Fist', 'SUBTriangle Armbar', 'M-DEC', 'DQ',
       'CNC', 'SUBAnaconda Choke', 'KO/TKOSlam', 'SUBNeck Crank',
       'Overturned', 'KO/TKOKnees', 'KO/TKOKicks', 'SUBEzekiel Choke',
       'SUBTwister', 'KO/TKOSpinning Back Elbow',
       'OverturnedGuillotine Choke', 'KO/TKOElbow', 'SUBOther - Lock',
       'SUBKneebar', 'SUBInverted Triangle', 'KO/TKOSpinning Back Kick',
       'SUBVon Flue Choke', 'SUBKeylock', 'SUBStraight Armbar',
       'SUBScarf Hold', 'SUBHeel Hook', 'OverturnedPunches',
       'SUBAnkle Lock', 'SUBForearm Choke', 'SUBInjury',
       'OverturnedPunch', 'OverturnedRear Naked Choke', 'OverturnedKnee',
       'OverturnedArm Triangle', 'SUBPeruvian Necktie',
 

In [493]:
# Filtering out unusual fight results
unusal_meths = ['CNC', 'M-DEC', 'DQ',  'OverturnedKick', 'DQRear Naked Choke', 'OverturnedTriangle Choke','OverturnedElbows', 'Other']
final_df2 = final_df[~final_df['method'].isin(unusal_meths)]

In [494]:
len(final_df2)

7464

In [495]:
complete_df = final_df2.merge(stats_df1, on = ['stats_url', 'fighter1', 'fighter2'], how="right")

In [487]:
complete_df.columns

Index(['event_x', 'fighter1', 'fighter2', 'weight', 'rounds', 'times',
       'method', 'locations', 'dates', 'stats_url', 'f1_kd', 'f1_sigstr',
       'f1_sigstr_pct', 'f1_totstr', 'f1_td', 'f1_td_pct', 'f1_subatt',
       'f1_rev', 'f1_ctrl', 'f2_kd', 'f2_sigstr', 'f2_sigstr_pct', 'f2_totstr',
       'f2_td', 'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'event_y',
       'key'],
      dtype='object')

In [496]:
next_join_table = complete_df[complete_df['event_x'].isnull()]

In [497]:
# Rows where fighter1 and fighter2 are switched. Switching columns names, concatenating rows to original df and rejoining data to produce final df
print(next_join_table)

     event_x           fighter1          fighter2 weight rounds times method  \
1        NaN      Anthony Smith    Zhang Mingyang    NaN    NaN   NaN    NaN   
2        NaN      Giga Chikadze       David Onama    NaN    NaN   NaN    NaN   
3        NaN     Michel Pereira    Abus Magomedov    NaN    NaN   NaN    NaN   
13       NaN   Chelsea Chandler  Joselyne Edwards    NaN    NaN   NaN    NaN   
19       NaN  Ronaldo Rodriguez      Kevin Borjas    NaN    NaN   NaN    NaN   
...      ...                ...               ...    ...    ...   ...    ...   
7339     NaN      Renato Sobral     Maurice Smith    NaN    NaN   NaN    NaN   
7357     NaN      Matt Lindland   Ricardo Almeida    NaN    NaN   NaN    NaN   
7365     NaN      Laverne Clark        Koji Oishi    NaN    NaN   NaN    NaN   
7377     NaN     Amaury Bitetti      Alex Andrade    NaN    NaN   NaN    NaN   
7403     NaN        Jens Pulver   Alfonso Alcarez    NaN    NaN   NaN    NaN   

     locations dates                   

In [498]:
next_join_table = next_join_table.rename(columns={
'fighter1' : 'fighter2',
'fighter2': 'fighter1',
'f1_kd' : 'f2_kd',
'f1_sigstr' : 'f2_sigstr',
'f1_sigstr_pct' : 'f2_sigstr_pct',
'f1_totstr' : 'f2_totstr',
'f1_td' : 'f2_td',
'f1_td_pct' : 'f2_td_pct',
'f1_subatt' : 'f2_subatt',
'f1_rev' : 'f2_rev',
'f1_ctrl' : 'f2_ctrl',
'f2_kd' : 'f1_kd',
'f2_sigstr' : 'f1_sigstr',
'f2_sigstr_pct' : 'f1_sigstr_pct',
'f2_totstr' : 'f1_totstr',
'f2_td' : 'f1_td',
'f2_td_pct' : 'f1_td_pct',
'f2_subatt' : 'f1_subatt',
'f2_rev' : 'f1_rev',
'f2_ctrl' : 'f1_ctrl',
'event_y' : 'event'
})

In [491]:
next_join_table.columns

Index(['fighter1', 'fighter2', 'weight', 'rounds', 'times', 'method',
       'locations', 'dates', 'stats_url', 'f1_kd', 'f1_sigstr',
       'f1_sigstr_pct', 'f1_totstr', 'f1_td', 'f1_td_pct', 'f1_subatt',
       'f1_rev', 'f1_ctrl', 'f2_kd', 'f2_sigstr', 'f2_sigstr_pct', 'f2_totstr',
       'f2_td', 'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'event'],
      dtype='object')

In [ ]:
# next_join_table = next_join_table.drop(['event_x', 'key'], axis=1)

KeyError: "['event_x', 'key'] not found in axis"

In [499]:
final_stats_join = pd.concat([next_join_table, stats_df1], ignore_index=True)

complete_df = final_df2.merge(final_stats_join, on = ['stats_url', 'fighter1', 'fighter2'], how="inner")
len(complete_df)

7287

In [500]:
complete_df.columns

Index(['event_x', 'fighter1', 'fighter2', 'weight_x', 'rounds_x', 'times_x',
       'method_x', 'locations_x', 'dates_x', 'stats_url', 'event_x',
       'weight_y', 'rounds_y', 'times_y', 'method_y', 'locations_y', 'dates_y',
       'f2_kd', 'f2_sigstr', 'f2_sigstr_pct', 'f2_totstr', 'f2_td',
       'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'f1_kd', 'f1_sigstr',
       'f1_sigstr_pct', 'f1_totstr', 'f1_td', 'f1_td_pct', 'f1_subatt',
       'f1_rev', 'f1_ctrl', 'event_y', 'key'],
      dtype='object')

In [501]:
complete_df = complete_df.drop(columns=[
'weight_y',	'rounds_y',	'times_y',	'method_y',	'locations_y',	'dates_y', 'event_y', 'key'

], axis=1)

In [ ]:
complete_df = complete_df.rename(columns={
'event_x' : 'event',
'weight_x': 'weight',
'rounds_x' : 'rounds',
'times_x' : 'times',
'method_x' : 'method',
'locations_x' : 'locations',
'dates_x' : 'dates'
})

In [503]:
complete_df[['f1_sigstr_landed', 'f1_sigstr_attempt']] = complete_df['f1_sigstr'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_sigstr_landed', 'f2_sigstr_attempt']] = complete_df['f2_sigstr'].str.split(' of ', expand=True).astype(int)

complete_df[['f1_totstr_landed', 'f1_totstr_attempt']] = complete_df['f1_totstr'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_totstr_landed', 'f2_totstr_attempt']] = complete_df['f2_totstr'].str.split(' of ', expand=True).astype(int)

complete_df[['f1_td_landed', 'f1_td_attempt']] = complete_df['f1_td'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_td_landed', 'f2_td_attempt']] = complete_df['f2_td'].str.split(' of ', expand=True).astype(int)

In [504]:
complete_df = complete_df.drop(columns=[
'f1_sigstr', 'f2_sigstr', 'f1_totstr', 'f2_totstr', 'f1_td', 'f2_td'
], axis=1)

In [505]:
complete_df['f1_sigstr_pct'] = complete_df.apply(lambda row: row['f1_sigstr_landed']/row['f1_sigstr_attempt'] * 100 if row['f1_sigstr_attempt'] != 0 else 0, axis=1)
complete_df['f2_sigstr_pct'] = complete_df.apply(lambda row: row['f2_sigstr_landed']/row['f2_sigstr_attempt'] * 100 if row['f2_sigstr_attempt'] != 0 else 0, axis=1)

complete_df['f2_totstr_pct'] = complete_df.apply(lambda row: row['f2_totstr_landed']/row['f2_totstr_attempt'] * 100 if row['f2_totstr_attempt'] != 0 else 0, axis=1)
complete_df['f1_totstr_pct'] = complete_df.apply(lambda row: row['f1_totstr_landed']/row['f1_totstr_attempt'] * 100 if row['f1_totstr_attempt'] != 0 else 0, axis=1)

complete_df['f1_td_pct'] = complete_df.apply(lambda row: row['f1_td_landed']/row['f1_td_attempt'] * 100 if row['f1_td_attempt'] != 0 else 0, axis=1)
complete_df['f2_td_pct'] = complete_df.apply(lambda row: row['f2_td_landed']/row['f2_td_attempt'] * 100 if row['f2_td_attempt'] != 0 else 0, axis=1)

In [506]:
complete_df['rounds'] = complete_df['rounds'].astype(int)

In [507]:
# Get MM:SS time format into just seconds to help aggregate / total time in the fight
def standard_time_format(mmss):
    if mmss == '--':
        return 0
    else: minutes, seconds = map(int, mmss.split(":"))
    return (minutes*60) + seconds

def total_fight_time(row):
    remainder = standard_time_format(row['times'])
    x = row['rounds'] - 1
    return (x*5*60) + remainder

In [508]:
complete_df['f1_ctrl_sec'] = complete_df['f1_ctrl'].apply(standard_time_format)
complete_df['f2_ctrl_sec'] = complete_df['f2_ctrl'].apply(standard_time_format)

In [479]:
complete_df['tot_fight_secs'] = complete_df.apply(total_fight_time, axis=1)

In [510]:
complete_df.columns

Index(['event', 'fighter1', 'fighter2', 'weight', 'rounds', 'times', 'method',
       'location', 'dates', 'stats_url', 'event', 'f2_kd', 'f2_sigstr_pct',
       'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'f1_kd', 'f1_sigstr_pct',
       'f1_td_pct', 'f1_subatt', 'f1_rev', 'f1_ctrl', 'f1_sigstr_landed',
       'f1_sigstr_attempt', 'f2_sigstr_landed', 'f2_sigstr_attempt',
       'f1_totstr_landed', 'f1_totstr_attempt', 'f2_totstr_landed',
       'f2_totstr_attempt', 'f1_td_landed', 'f1_td_attempt', 'f2_td_landed',
       'f2_td_attempt', 'f2_totstr_pct', 'f1_totstr_pct', 'f1_ctrl_sec',
       'f2_ctrl_sec'],
      dtype='object')

In [512]:
complete_df['more_totstr_landed'] = complete_df.apply(lambda row: 'fighter1' if row['f1_totstr_landed'] > row['f2_totstr_landed'] else ('fighter2' if row['f1_totstr_landed'] < row['f2_totstr_landed'] else "equal"), axis=1)
complete_df['more_totstr_attempt'] = complete_df.apply(lambda row: 'fighter1' if row['f1_totstr_attempt'] > row['f2_totstr_attempt'] else ('fighter2' if row['f1_totstr_attempt'] < row['f2_totstr_attempt'] else "equal"), axis=1)

complete_df['more_sigstr_attempt'] = complete_df.apply(lambda row: 'fighter1' if row['f1_sigstr_attempt'] > row['f2_sigstr_attempt'] else ('fighter2' if row['f1_sigstr_attempt'] < row['f2_sigstr_attempt'] else "equal"),axis=1)
complete_df['more_sigstr_landed'] = complete_df.apply(lambda row: 'fighter1' if row['f1_sigstr_landed'] > row['f2_sigstr_landed'] else ('fighter2' if row['f1_sigstr_landed'] < row['f2_sigstr_landed'] else "equal"),axis=1)


In [ ]:
complete_df.to_csv("Complete Stats.csv")

In [56]:
stats_df = pd.read_csv("Complete Stats.csv")

# separate into two tables then append each other to normalize the data
# stats_df.columns
f1_df = stats_df[['event', 'fighter1', 'weight', 'rounds',
       'times', 'method', 'location', 'dates', 'stats_url', 'f1_kd',
       'f1_sigstr_pct', 'f1_td_pct', 'f1_subatt', 'f1_rev', 'f1_ctrl',
       'f1_sigstr_landed', 'f1_sigstr_attempt', 'f1_totstr_landed', 'f1_totstr_attempt', 'f1_td_landed',
       'f1_td_attempt',
       'f1_totstr_pct', 'f1_ctrl_sec', 'more_totstr_landed',
       'more_totstr_attempt', 'more_sigstr_attempt', 'more_sigstr_landed']].copy()

f1_df["is_winner"] = True
f1_df["fighter_num"] = "fighter1"
f1_df.rename(columns={"fighter1" : "fighter" , 'f1_kd' : 'kd',
       'f1_sigstr_pct' : 'sigstr_pct', 'f1_td_pct' : 'td_pct', 'f1_subatt' : 'subatt', 'f1_rev' : 'rev', 'f1_ctrl' : 'ctrl', 'f1_sigstr_landed' : 'sigstr_landed',
       'f1_sigstr_attempt' : 'sigstr_attempt',
       'f1_totstr_landed' : 'totstr_landed', 'f1_totstr_attempt' : 'totstr_attempt', 'f1_td_landed' : 'td_landed', 'f1_td_attempt' : 'td_attempt', 'f1_totstr_pct' : 'totstr_pct', 'f1_ctrl_sec' : 'ctrl_sec'}, inplace=True)

f2_df = stats_df[['event', 'fighter2', 'weight', 'rounds',
       'times', 'method', 'location', 'dates', 'stats_url', 'f2_kd',
       'f2_sigstr_pct', 'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'f2_sigstr_landed',
       'f2_sigstr_attempt',
       'f2_totstr_landed', 'f2_totstr_attempt', 'f2_td_landed', 'f2_td_attempt', 'f2_totstr_pct', 'f2_ctrl_sec', 'more_totstr_landed',
       'more_totstr_attempt', 'more_sigstr_attempt', 'more_sigstr_landed']].copy()

f2_df["is_winner"] = False
f2_df["fighter_num"] = "fighter2"
f2_df.rename(columns={"fighter2" : "fighter", "f2_kd": "kd",
       'f2_sigstr_pct': 'sigstr_pct', 'f2_td_pct': 'td_pct', 'f2_subatt' : 'subatt', 'f2_rev': 'rev', 'f2_ctrl': 'ctrl', 'f2_sigstr_landed': 'sigstr_landed',
       'f2_sigstr_attempt' : 'sigstr_attempt',
       'f2_totstr_landed' : 'totstr_landed', 'f2_totstr_attempt' : 'totstr_attempt', 'f2_td_landed' : 'td_landed', 'f2_td_attempt' : 'td_attempt', 'f2_totstr_pct' : 'totstr_pct', 'f2_ctrl_sec' : 'ctrl_sec'}, inplace=True)

In [58]:
norm_df = pd.concat([f1_df, f2_df], ignore_index = True)

In [59]:
norm_df.to_csv("Normalized Stats Table.csv")